In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('dataset copy.csv')

In [3]:
def weight_mapping(weight_text):
    if weight_text == "저체중":
        return -2.5
    elif weight_text == "정상":
        return 0
    elif weight_text == "비만전단계비만":
        return 1
    elif weight_text == "1단계비만":
        return 2
    elif weight_text == "2단계비만":
        return 3
    elif weight_text == "3단계비만":
        return 4

In [4]:
def sex_mapping(sex):
    if sex == "F":
        return -1
    if sex == "M":
        return 1

In [5]:
header = {}
for i in (list(data['RECOMEND_MVM_NM'])):
    
    if ( "동적" in i): continue
    if ( "스트레칭" in i): continue
    
    if i not in header:
        header[i] = 0
    
    header[i] += 1

In [6]:
# Sort the dictionary by value
header = dict(sorted(header.items(), key=lambda item: item[1], reverse=True))

# Get the top 10 sports
top_10_sports = list(header.keys())[:10]

filtered_data = data[data['RECOMEND_MVM_NM'].isin(top_10_sports)]

In [7]:
filtered_data

,AGRDE_FLAG_NM,BMI_IDEX_GRAD_NM,MBER_SEXDSTN_FLAG_CD,COAW_FLAG_NM,SPORTS_STEP_NM,FLAG_ACCTO_RECOMEND_MVM_RANK_CO,RECOMEND_MVM_NM
6,10대,정상,F,참가증,준비운동,2,몸통 비틀기
21,10대,정상,F,1등급,준비운동,2,몸통 비틀기
36,10대,정상,F,2등급,준비운동,2,몸통 비틀기
51,10대,정상,F,3등급,준비운동,2,몸통 비틀기
67,10대,정상,M,참가증,준비운동,3,몸통 비틀기
...,...,...,...,...,...,...,...
4998,70대 이상,3단계비만,M,1등급,본운동,4,엎드려 양팔 및 다리 들어올리기
5010,70대 이상,3단계비만,M,2등급,본운동,1,몸통 들어올리기
5014,70대 이상,3단계비만,M,2등급,본운동,5,엎드려 양팔 및 다리 들어올리기
5025,70대 이상,3단계비만,M,3등급,본운동,1,몸통 들어올리기


In [8]:
top_10_sports

['실내 자전거타기',
 '달리기',
 '다리 벌려 앞으로 상체 숙이기',
 '앉았다 일어서기',
 '버피운동',
 '몸통 비틀기',
 '다리 벌려 옆으로 상체 숙이기',
 '발목 얹고 다리 잡아당기기',
 '엎드려 양팔 및 다리 들어올리기',
 '몸통 들어올리기']

In [9]:
'''[
 '실내 자전거타기',
 '달리기',
 '다리 벌려 앞으로 상체 숙이기',
 '앉았다 일어서기',
 '버피운동',
 '몸통 비틀기',
 '다리 벌려 옆으로 상체 숙이기',
 '발목 얹고 다리 잡아당기기',
 '엎드려 양팔 및 다리 들어올리기',
 '몸통 들어올리기'
 ]'''
 
def one_hot_encoder(exercise:str):
    if exercise == "실내 자전거타기":
        return [1,0,0,0,0,0,0,0,0,0]
    elif exercise == "달리기":
        return [0,1,0,0,0,0,0,0,0,0]
    elif exercise == "다리 벌려 앞으로 상체 숙이기":
        return [0,0,1,0,0,0,0,0,0,0]
    elif exercise == "앉았다 일어서기":
        return [0,0,0,1,0,0,0,0,0,0]
    elif exercise == "버피운동":
        return [0,0,0,0,1,0,0,0,0,0]
    elif exercise == "몸통 비틀기":
        return [0,0,0,0,0,1,0,0,0,0]
    elif exercise == "다리 벌려 옆으로 상체 숙이기":
        return [0,0,0,0,0,0,1,0,0,0]
    elif exercise == "발목 얹고 다리 잡아당기기":
        return [0,0,0,0,0,0,0,1,0,0]
    elif exercise == "엎드려 양팔 및 다리 들어올리기":
        return [0,0,0,0,0,0,0,0,1,0]
    elif exercise == "몸통 들어올리기":
        return [0,0,0,0,0,0,0,0,0,1]

    return [0,0,0,0,0,0,0,0,0,0]

def one_hot_decoder(result):
    if result == 1:
        return "실내 자전거타기"
    
    if result == 2:
        return "달리기"
    
    if result == 3:
        return "다리 벌려 앞으로 상체 숙이기"
    
    if result == 4:
        return "앉았다 일어서기"
    
    if result == 5:
        return "버피운동"
    
    if result == 6:
        return "몸통 비틀기"
    
    if result == 7:
        return "다리 벌려 옆으로 상체 숙이기"
    
    if result == 8:
        return "발목 얹고 다리 잡아당기기"
    
    if result == 9:
        return "엎드려 양팔 및 다리 들어올리기"
    
    if result == 10:
        return "몸통 들어올리기"
    

In [10]:
import torch

trainset = []

for index, row in filtered_data.iterrows():
    
    trainset.append({
        'data': torch.tensor([
            sex_mapping(row["MBER_SEXDSTN_FLAG_CD"]),
            int(row['AGRDE_FLAG_NM'].replace("대","").replace(" 이상","")),
            weight_mapping(row['BMI_IDEX_GRAD_NM']),
        ]).float(),
        'target': torch.tensor(one_hot_encoder(row['RECOMEND_MVM_NM'])).float()
    })
    
        

In [11]:
import torch
import torch.nn as nn

In [12]:
class DNN(nn.Module):
    
    def __init__(self):
        super(DNN, self).__init__()
        
        self.fc1 = nn.Linear(3, 40)
        self.activation = nn.LeakyReLU()
        self.fc2 = nn.Linear(40, 10)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [14]:
import random

model = DNN()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

epoch = 1000

for i in range(epoch):
    
    # Reset the gradient
    optimizer.zero_grad()
    
    for this in random.sample(trainset, 100):
        output = model(this['data'])
        loss = criterion(output, this['target'])
        loss.backward()
        
    optimizer.step()    
    print(f"Epoch {i+1}/{epoch} Loss: {loss.item()}")

Epoch 1/1000 Loss: 2.274643659591675
Epoch 2/1000 Loss: 2.458411455154419
Epoch 3/1000 Loss: 2.4566574096679688
Epoch 4/1000 Loss: 2.454493522644043
Epoch 5/1000 Loss: 2.4226887226104736
Epoch 6/1000 Loss: 2.4570484161376953
Epoch 7/1000 Loss: 2.454162359237671
Epoch 8/1000 Loss: 2.4465010166168213
Epoch 9/1000 Loss: 2.4308314323425293
Epoch 10/1000 Loss: 1.740230917930603
Epoch 11/1000 Loss: 2.430098533630371
Epoch 12/1000 Loss: 2.43275785446167
Epoch 13/1000 Loss: 2.42659854888916
Epoch 14/1000 Loss: 2.4265336990356445
Epoch 15/1000 Loss: 2.4250071048736572
Epoch 16/1000 Loss: 2.423214912414551
Epoch 17/1000 Loss: 1.801675796508789
Epoch 18/1000 Loss: 2.425138235092163
Epoch 19/1000 Loss: 2.42806339263916
Epoch 20/1000 Loss: 2.423123359680176
Epoch 21/1000 Loss: 2.4268417358398438
Epoch 22/1000 Loss: 1.972914695739746
Epoch 23/1000 Loss: 1.8559824228286743
Epoch 24/1000 Loss: 2.0549235343933105
Epoch 25/1000 Loss: 2.424741506576538
Epoch 26/1000 Loss: 2.4245033264160156
Epoch 27/1000

In [15]:
model.eval()

# Test the model
correct = 0
total = 0
one_hot_decoder(model(torch.tensor([1, 30, 0]).float()).argmax().item())

torch.save(model.state_dict(), 'model.pth')